In [1]:
pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /private/var/folders/zn/c3p45tzj01g5kzxf3374t0lw0000gn/T/pip-req-build-fqeo73ez
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /private/var/folders/zn/c3p45tzj01g5kzxf3374t0lw0000gn/T/pip-req-build-fqeo73ez
  Resolved https://github.com/openai/whisper.git to commit e8622f9afc4eba139bf796c210f5c01081000472
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done

[notice] A new release of pip is available: 23.1.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install openai


[notice] A new release of pip is available: 23.1.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


**Extracting audio from mp4 file**

In [3]:
import subprocess
command = 'ffmpeg -i video1.mp4 -ab 160k -ar 44100 -vn audio.wav'
subprocess.call(command, shell=True)

ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/6.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack --dis

KeyboardInterrupt: 

**High-level audio transcription method**

In [15]:
import whisper
model = whisper.load_model('base')
result = model.transcribe('video1.mp4', fp16=False)

In [19]:
transcript = result["text"]

In [23]:
print("Does the following text contain any misinformation about the vaccine? : " + transcript)

Does the following text contain any misinformation about the vaccine? :  Christians don't get to vaccine at all. P Pfizer, BioNTech, COVID-19 vaccine FDA. All right, BioNTech. German biotechnology company. What does biotechnology mean? This technology that utilizes biological systems, living organisms or parts of this to create different products make changes to the organism's genetic material. This is what the following angels were doing. This is what people were doing pre-flood. If you get the biore vaccine, you're going along with the manipulation of DNA and that is unacceptable to God. Period. He destroyed the world once for it. He's going to do it again. Don't get to vaccine.


**GPT Classifier**

In [26]:
import os
import openai

openai.api_key = ""

response = openai.Completion.create(
  model="text-davinci-003",
  prompt= "Does the following text contain any misinformation about the vaccine? : " + transcript,
  temperature=0,
  max_tokens=64,
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=0.0
)

RateLimitError: You exceeded your current quota, please check your plan and billing details.

In [23]:
print(response)

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n\nYes, this text contains misinformation about the vaccine. It implies that Christians should not get the vaccine because it involves manipulating DNA, which is not true. The vaccine does not involve manipulating DNA."
    }
  ],
  "created": 1687452497,
  "id": "cmpl-7UHlhAbrwK5WW9eNi5mok9b5riPUW",
  "model": "text-davinci-003",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 40,
    "prompt_tokens": 147,
    "total_tokens": 187
  }
}


**Low-level audio transcription method**

In [12]:
low_model = whisper.load_model('base')

#load audio and pad or trim to fit 30 seconds
audio = whisper.load_audio('audio.wav')
audio = whisper.pad_or_trim(audio)

#make log-Mel spectrogram and move to the same device as the model
mel = whisper.log_mel_spectrogram(audio).to(model.device)

#detect the spoken langauge
_, probs = model.detect_language(mel)
print(f"Detected language: {max(probs, key=probs.get)}")

#decode the audio
options = whisper.DecodingOptions(fp16 = False)
result = whisper.decode(model, mel, options)

#print the recognized text
print(result.text)

AttributeError: 'DecodingOptions' object has no attribute 'text'

**OpenAI vs Google Speech Recognition API**

In [6]:
import speech_recognition as sr

#Initialize the recognizer
r = sr.Recognizer()
with sr.AudioFile('audio.wav') as source:
    audio_text = r.listen(source)
r.recognize_google(audio_text, show_all=False)

'the new health insurance policy for the playoffs about me the best ability is availability adalo no'